<a href="https://colab.research.google.com/github/Asuskf/intro-aprendizaje-por-transferencia/blob/main/Flisol2021_Intro_aprendizaje_por_transferencia_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/joanby/tensorflow2/blob/master/Collab%206%20-%20Aprendizaje%20por%20Transferencia%20y%20puesta%20a%20punto%20de%20parámetros%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://live.staticflickr.com/4544/38228876666_3782386ca7_b.jpg)

## Paso 1: Instalación de dependencias y configuración del entorno en GPU

In [ ]:
#!pip install tensorflow-gpu==2.0.0.alpha0
%tensorflow_version 2.x

In [ ]:
!pip install tqdm

### Descargar el dataset de Perros vs Gatos

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2021-04-13 16:27:39--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.7.240, 172.217.8.16, 172.217.13.80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.7.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   194MB/s    in 0.3s    

2021-04-13 16:27:40 (194 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## Paso 2: Pre procesado del data set

### Importar las dependencias del proyecto

In [ ]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.4.1'

### Descomprimir el dataset de Perros vs Gatos

In [ ]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [ ]:
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")

In [ ]:
zip_object.extractall("./")

In [ ]:
zip_object.close()

### Configurar las rutas al dataset

In [ ]:
dataset_path_new = "./cats_and_dogs_filtered/"

In [ ]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

## Construir el Modelo

### Cargar un modelo pre entrenado (MobileNetV2)

In [ ]:
IMG_SHAPE = (128, 128, 3)

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

9412608/9406464 [==============================] - 0s 0us/step


In [ ]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 64, 64, 32)   0           bn_Conv1[0][0]                   
_______________________________________________________________________________

### Congelar el modelo base

In [ ]:
base_model.trainable = False

### Definir la cabecera personalizada para nuestra red neuronal

In [ ]:
base_model.output

<KerasTensor: shape=(None, 4, 4, 1280) dtype=float32 (created by layer 'out_relu')>

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [ ]:
global_average_layer

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d')>

In [ ]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

### Definir el modelo

In [ ]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 64, 64, 32)   0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

### Compilar el modelo

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

### Crear generadores de datos

Redimensionar imágenes

    Las grandes arquitecturas pre-entrenadas solamente soportan cierto tipo de tamaños de imágenes.
    
Por ejemplo: MobileNet (la arquitectura que nosotros usamos) soporta: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).

In [ ]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [ ]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.


In [ ]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 1000 images belonging to 2 classes.


### Entrenar el modelo 

In [ ]:
model.fit_generator(train_generator, epochs=8, validation_data=valid_generator)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/8
16/16 [==============================] - 43s 578ms/step - loss: 0.8274 - accuracy: 0.5077 - val_loss: 0.7229 - val_accuracy: 0.5810
Epoch 2/8
16/16 [==============================] - 7s 481ms/step - loss: 0.6562 - accuracy: 0.6211 - val_loss: 0.6140 - val_accuracy: 0.6740
Epoch 3/8
16/16 [==============================] - 7s 471ms/step - loss: 0.5673 - accuracy: 0.7197 - val_loss: 0.5305 - val_accuracy: 0.7620
Epoch 4/8
16/16 [==============================] - 7s 474ms/step - loss: 0.4860 - accuracy: 0.8018 - val_loss: 0.4621 - val_accuracy: 0.8290
Epoch 5/8
16/16 [==============================] - 7s 473ms/step - loss: 0.4276 - accuracy: 0.8360 - val_loss: 0.4059 - val_accuracy: 0.8640
Epoch 6/8
16/16 [==============================] - 8s 478ms/step - loss: 0.3877 - accuracy: 0.8449 - val_loss: 0.3609 - val_accuracy: 0.8950
Epoch 7/8
16/16 [==============================] - 8s 483ms/step - loss: 0.3356 - accuracy: 0.8802 - val_loss: 0.3234 - val_accuracy: 0.9020
Epoch 8/8
16

### Evaluar el modelo de aprendizaje por transferencia

In [ ]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


In [ ]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.9150000214576721


## Puesta a punto de parámetros


Un par de cosas:

- NUNCA HAY QUE USAR la puesta a punto (fine tuning) de parámetros en toda la red neuronal: con algunas de las capas superiores (las finales) es más que suficiente suficiente. En la mayoría de casos, son las más especializadas. El objetivo del fine tuning es adaptar esa parte específica de la red neuronal para nuestro nuevo dataset específico.
- Empezar con la puesta a punto DESPUÉS de haber finalizado la fase de aprendizaje por transferencia. Si intentamos hacer el Fine tuning inmediatamente, los gradientes serán muy diferentes entre nuestra cabecera personalizada de la red neuronal y las nuevas capas no congeladas del modelo base.

### Descongelar unas cuantas capas superiores del modelo

In [ ]:
base_model.trainable = True

In [ ]:
print("Number of layersin the base model: {}".format(len(base_model.layers)))

Number of layersin the base model: 154


In [ ]:
fine_tune_at = 100

In [ ]:
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

### Compilar el modelo para la puesta a punto

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

### Puesta a punto

In [ ]:
model.fit_generator(train_generator,  
                    epochs=5, 
                    validation_data=valid_generator)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
16/16 [==============================] - 13s 551ms/step - loss: 0.2704 - accuracy: 0.8698 - val_loss: 0.0923 - val_accuracy: 0.9630
Epoch 2/5
16/16 [==============================] - 8s 494ms/step - loss: 0.0292 - accuracy: 0.9975 - val_loss: 0.0989 - val_accuracy: 0.9660
Epoch 3/5
16/16 [==============================] - 8s 493ms/step - loss: 0.0101 - accuracy: 0.9999 - val_loss: 0.1009 - val_accuracy: 0.9660
Epoch 4/5
16/16 [==============================] - 8s 486ms/step - loss: 0.0038 - accuracy: 1.0000 - val_loss: 0.1179 - val_accuracy: 0.9640
Epoch 5/5
16/16 [==============================] - 8s 487ms/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.2417 - val_accuracy: 0.9500


### Evaluar el modelo re calibrado

In [ ]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


In [ ]:
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))

Validation accuracy after fine tuning: 0.949999988079071
